### Australian Rainfall Data Acquisition and EDA

This notebook will be a quick run through of data acquisition and exploratory analysis to evaluate the impact of big data tools. These tools include Dask for out of core processing and Feather/Arrow/Parquet file formats to minimize memory usage and file size. 

In [1]:
import re
import io
import os
import glob
import requests
import json
from urllib.request import urlretrieve
import zipfile
import pandas as pd
import numpy as np
import dask.dataframe as dd

# Get helper functions
from scripts.utils import combine_australia_rainfall

%load_ext rpy2.ipython
%load_ext memory_profiler
%load_ext autoreload


# Constants
raw_data_directory_path = os.path.join("..", "data", "raw")

C:\Users\Dustin\miniconda3\envs\525_group10\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


First, download the zipped archive folder of all rainfall data - NOTE: THIS CAN TAKE ~ 45 minutes to run:

In [ ]:
def download_progress(block_num, block_size, total_size):
    progress_size = int(block_num * block_size)
    percent = int(block_num * block_size * 100 / total_size)
    sys.stdout.write("\rDownloading... %d%%, %d MB of %d MB" %
                    (percent, progress_size / (1024 * 1024), total_size / (1024 * 1024)))
    sys.stdout.flush()


def download_from_figshare(article_id, files_to_dl, output_dir):
    url = f"https://api.figshare.com/v2/articles/{article_id}"
    response = requests.request("GET", url, headers={"Content-Type": "application/json"})
    data = json.loads(response.text)
    files = data["files"]
    for file in files:
        if file["name"] in files_to_dl:
            os.makedirs(output_dir, exist_ok=True)
            urlretrieve(file["download_url"], os.path.join(output_dir, file["name"]), download_progress)


download_from_figshare("14096681", ["data.zip"], raw_data_directory_path)

Next, iterate through the zipped files, extracting the raw csv files:

In [8]:
with zipfile.ZipFile(os.path.join(raw_data_directory_path, "data.zip"), "r") as zf:
    file_list = zf.namelist()
    for f in file_list:
        if "__MACOSX" not in f:
            print(f"Processing File: {f}")
            zf.extract(f, path = raw_data_directory_path)


Processing File: MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
Processing File: AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
Processing File: NorESM2-LM_daily_rainfall_NSW.csv
Processing File: ACCESS-CM2_daily_rainfall_NSW.csv
Processing File: FGOALS-f3-L_daily_rainfall_NSW.csv
Processing File: CMCC-CM2-HR4_daily_rainfall_NSW.csv
Processing File: MRI-ESM2-0_daily_rainfall_NSW.csv
Processing File: GFDL-CM4_daily_rainfall_NSW.csv
Processing File: BCC-CSM2-MR_daily_rainfall_NSW.csv
Processing File: EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
Processing File: CMCC-ESM2_daily_rainfall_NSW.csv
Processing File: NESM3_daily_rainfall_NSW.csv
Processing File: MPI-ESM1-2-LR_daily_rainfall_NSW.csv
Processing File: ACCESS-ESM1-5_daily_rainfall_NSW.csv
Processing File: FGOALS-g3_daily_rainfall_NSW.csv
Processing File: INM-CM4-8_daily_rainfall_NSW.csv
Processing File: MPI-ESM1-2-HR_daily_rainfall_NSW.csv
Processing File: TaiESM1_daily_rainfall_NSW.csv
Processing File: NorESM2-MM_daily_rainfall_NSW.csv
Processing File:

Next we'll evaluate the impact of combining multiple large CSV files using Pandas vs. Dask. We'll check how fast Dask is when creating the task graph as well as materializing to a Pandas dataframe.

In [3]:
%%time
%%memit
df_rainfall_pandas = combine_australia_rainfall(base_folder_path=raw_data_directory_path)

C:\Users\Dustin\miniconda3\envs\525_group10\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
peak memory: 14896.05 MiB, increment: 14751.36 MiB
Wall time: 6min 19s


In [ ]:
%%time
%%memit
df_rainfall_pandas = combine_australia_rainfall(base_folder_path=raw_data_directory_path)

In [4]:
%%time
%%memit
df_rainfall_dask_delay = combine_australia_rainfall(base_folder_path=raw_data_directory_path, method="dask", delay_dask_compute=True)

peak memory: 10600.93 MiB, increment: 7.31 MiB
Wall time: 2.86 s


In [5]:
%%time
%%memit
combine_australia_rainfall(base_folder_path=raw_data_directory_path, method="dask", delay_dask_compute=False)

peak memory: 18174.11 MiB, increment: 7580.56 MiB
Wall time: 7min 11s


In [ ]:
%%time
%%memit
combine_australia_rainfall(base_folder_path=raw_data_directory_path, method="dask", delay_dask_compute=False)

Looks like using Dask is ~ 50% faster to read and concatentate multiple large CSV files.

Next, we'll look at the speed of determining counts of values by group from the raw dataframe.

In [6]:
%%time
%%memit
df_rainfall_pandas["model"].value_counts()


peak memory: 13896.96 MiB, increment: 379.98 MiB
Wall time: 43.5 s


In [7]:
%%time
%%memit
model_counts = df_rainfall_dask_delay["model"].value_counts().compute()

peak memory: 15555.75 MiB, increment: 1717.35 MiB
Wall time: 3min 50s


In [ ]:
df_pandas_rainfall.head()

### Challenges Encountered

Working with larger data sets on laptop can be difficult due to memory restrictions when loading a dataset. In the exercises in this lab we ran into difficulties with:
- Downloading large zip archives:
    - Getting the initial data downloaded was difficult as we were downloading one zip archive, and we couldn't parallelize this process. If there had been multiple files we could have used `multiprocessing` to download much quicker.
- Combining the dataset / EDA:
    - When combining the files with `Pandas` we were loading the entire dataset to memory and this caused issues for some group members with lower RAM. If we knew in advance what calculations we wanted to carry out, we could have delayed execution and used Dask to build the graph and then run an optimized `compute()` step at the end. This can be seen above when combining all files and calculating counts of readings per model using lazy evaluation.